## Expectations

**Functions**

`np.random.RandomState`, `RandomState.standard_normal`, `RandomState.standard_t`, `RandomState.chi2`,
`np.exp`, `np.mean`, `np.std`, `scipy.integrate.quadrature`, `scipy.integrate.quad`

### Exercise 10

Compute $E\left[X\right]$, $E\left[X^{2}\right]$, $V\left[X\right]$ and the kurtosis of $X$ using Monte Carlo integration when $X$ is distributed:

1. Standard Normal
2. $N\left(0.08,0.2^{2}\right)$
3. Students $t_{8}$
4. $\chi_{5}^{2}$



In [1]:
import numpy as np

rs = np.random.RandomState(30092019)

reps = 10000

x = rs.standard_normal(reps)
mu = x.mean()
mu2 = (x**2).mean()
var = mu2 - mu**2
kurt = np.mean(((x - mu) ** 4)) / var**2

print(f"E[X] = {mu}, E[X**2]={mu2}, V[X]={var}, K[X]={kurt}")

E[X] = 0.01695855531609172, E[X**2]=0.9893481101158448, V[X]=0.9890605175174358, K[X]=3.008212429714876


In [4]:
x = rs.standard_t(8, size=reps)
mu = x.mean()
mu2 = (x**2).mean()
var = mu2 - mu**2
kurt = np.mean(((x - mu) ** 4)) / var**2

print(f"E[X] = {mu}, E[X**2]={mu2}, V[X]={var}, K[X]={kurt}")

E[X] = -0.01354509569000196, E[X**2]=1.2987568306632953, V[X]=1.298573361046044, K[X]=4.2291706282496655


In [3]:
x = rs.chisquare(5, size=reps)
mu = x.mean()
mu2 = (x**2).mean()
var = mu2 - mu**2
kurt = np.mean(((x - mu) ** 4)) / var**2

print(f"E[X] = {mu}, E[X**2]={mu2}, V[X]={var}, K[X]={kurt}")


E[X] = 5.070367506398907, E[X**2]=36.11392404179839, V[X]=10.405297391852521, K[X]=5.14344036255509


In [ ]:
x = rs.normal(0.08, 0.2, size=reps)
mu = x.mean()
mu2 = (x**2).mean()
var = mu2 - mu**2
kurt = np.mean(((x - mu) ** 4)) / var**2

print(f"E[X] = {mu}, E[X**2]={mu2}, V[X]={var}, K[X]={kurt}")

In [5]:
def expectations(x):
    mu = x.mean()
    mu2 = (x**2).mean()
    var = mu2 - mu**2
    kurt = np.mean(((x - mu) ** 4)) / var**2
    
    print(f"E[X] = {mu}, E[X**2]={mu2}, V[X]={var}, K[X]={kurt}")
reps = 1000000
expectations(rs.chisquare(5, reps))


E[X] = 5.002214815620888, E[X**2]=35.04526122674316, V[X]=10.023108165126047, K[X]=5.391230855086581


### Exercise 11 

1. Compute $E\left[\exp\left(X\right)\right]$ when $X\sim N\left(0.08,0.2^{2}\right)$.
2. Compare this to the analytical result for a Log-Normal random variable.


In [15]:
x = rs.normal(0.08, 0.2, size=reps)
mu_exp = np.mean(np.exp(x))
print(f"E[exp(X)]={mu_exp}")

E[exp(X)]=1.1052365499422352


In [16]:
analytical = np.exp(0.08 + 0.2**2/2)
print(f"Analytical = {analytical}")

Analytical = 1.1051709180756477


### Exercise 12

Explore the role of uncertainty in Monte Carlo integration by increasing the number of simulations 300% relative to the base case.


In [18]:
base_reps = 10000

x = rs.standard_normal((base_reps,100))
mus = x.mean(0)
std = mus.std()

x = rs.standard_normal((4*base_reps,100))
mus_4 = x.mean(0)
std_4 = mus_4.std()

print(f"Std. Dev (base): {std}")
print(f"Std. Dev (4*base): {std_4}")
print(f"Ratio: {std/std_4}")


Std. Dev (base): 0.010790126262401096
Std. Dev (4*base): 0.004946123382997521
Ratio: 2.1815319649106506


### Exercise 13

Compute the expectation in exercise 11 using quadrature.

**Note**: This requires writing a function which will return $\exp\left(x\right)\times\phi\left(x\right)$ where $\phi\left(x\right)$ is the pdf evaluated at $x$.

In [23]:
import scipy.stats as stats
from scipy.integrate import quadrature, romberg
def f(x):
    return np.exp(x) * stats.norm.pdf(x, 0.08, 0.2)

res, err = quadrature(f, -5 * 0.2, 5 * 0.2)
print(f"Quadrature: {res}")

Quadrature: 1.1051649239632353


In [24]:
res = romberg(f, -5 * 0.2, 5 * 0.2)
print(f"Quadrature (romberg): {res}")

Quadrature (romberg): 1.1051649244195154


### Exercise 14 

**Optional** (Much more challenging)

Suppose log stock market returns are distributed according to a Students t with 8 degrees of
freedom, mean 8% and volatility 20%. Utility maximizers hold a portfolio consisting of a
risk-free asset paying 1% and the stock market. Assume that they are myopic and only care
about next period wealth, so that 

$$U\left(W_{t+1}\right)=U\left(\exp\left(r_{p}\right)W_{t}\right)$$

and that $U\left(W\right)=\frac{W^{1-\gamma}}{1-\gamma}$ is CRRA with risk aversion $\gamma$.
The portfolio return is $r_{p}=wr_{s}+\left(1-w\right)r_{f}$ where $s$ is for stock market
and $f$ is for risk-free. A 4th order expansion of this utility around the expected wealth
next period is

$$E_{t}\left[U\left(W_{t+1}\right)\right]\approx\phi_{0}+\phi_{1}\mu_{1}^{\prime}+\phi_{2}\mu_{2}^{\prime}+\phi_{3}\mu_{3}^{\prime}+\phi_{4}\mu_{4}^{\prime}$$

where

$$\phi_{j}=\left(j!\right)^{-1}U^{\left(j\right)}\left(E_{t}\left[W_{t+1}\right]\right),$$

$$U^{(j)}=\frac{\partial^{j}U}{\partial W^{j}},$$

$$\mu_{k}^{\prime}=E_{t}\left[\left(r-\mu\right)_{p}^{k}\right],$$

and $\mu=E_{t}\left[r_{p}\right]$. Use Monte Carlo integration to examine how the weight in
the stock market varies as the risk aversion varies from 1.5 to 10. Note that when $\gamma=1$, $U\left(W\right)=\ln\left(W\right)$.
Use $W_{t}=1$ without loss of generality since the portfolio problem is homogeneous of degree 0 in wealth.